<a href="https://colab.research.google.com/github/Rogerio-mack/data-engineering/blob/main/how_map_reducer_sum_hadoop_streaming_on_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hadoop Streaming on Colab**: sum  

Vamos experimentar o **Hadoop Streaming**. O utilitário Hadoop Streaming essencialmente inicia um job MapReduce utilizando a biblioteca Hadoop Streaming. Essa biblioteca permite que você utilize scripts externos (como scripts em Python, Ruby ou outros) como mapper e reducer em um job MapReduce. Vamos utilizar isso para executar um programa para cálculo de soma de valores em um arquivo.  

O Hadoop Streaming emula mini-cluster Hadoop em um computador local. Não são criados NameNodes e DataNodes reais. Em vez disso, o Hadoop Streaming cria estruturas de dados em memória para simular o funcionamento de um cluster. Isso tem uma grande limitação para processamento, mas pode ser útil para teste, desenvolvimento e pequenos conjuntos de dados em que se deseja fazer processos de map-reduce, nem sempre fáceis de serem desenvolvidos e testados.

## JAVA

Hadoop é uma plataforma de software em Java de computação distribuída. Assim, para executar no Colab precisamos do Java instalado.


In [ ]:
# Versão do Java instalada
!java -version

openjdk version "11.0.24" 2024-07-16
OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04)
OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)


In [ ]:
!ls -l /usr/lib/jvm

total 4
lrwxrwxrwx 1 root root   21 Jul 21 20:53 java-1.11.0-openjdk-amd64 -> java-11-openjdk-amd64
drwxr-xr-x 9 root root 4096 Sep  9 13:18 java-11-openjdk-amd64


In [ ]:
# Instalando o Java
#
# Este script foi atualizado com a versão 11 do Java, mas deixe isso de alerta caso precise empregar outras versões por compatibilidade do hadoop/spark
#
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Criando variáveis de ambiente
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre"
# os.environ["PATH"] += ":$JAVA_HOME/bin:$JRE_HOME/bin:$HADOOP_HOME/bin:$HADOOP_HOME/sbin"

In [ ]:
import os

# Criando variáveis de ambiente
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.11.0-openjdk-amd64"
os.environ["JRE_HOME"] = "/usr/lib/jvm/java-1.11.0-openjdk-amd64/jre"
os.environ["PATH"] += ":$JAVA_HOME/bin:$JRE_HOME/bin:$HADOOP_HOME/bin:$HADOOP_HOME/sbin"

# Instalando o Hadoop

In [ ]:
# Downloading Hadoop 3.3.5
!wget -q https://dlcdn.apache.org/hadoop/common/hadoop-3.3.5/hadoop-3.3.5.tar.gz

# Extraindo
!tar -xzvf hadoop-3.3.5.tar.gz

# Copiar o ficheiro hadoop para user/local
!cp -r hadoop-3.3.5/ /usr/local/

# Caminho padrão de Java
!readlink -f /usr/bin/java | sed "s:bin/java::"

# Hadoop
!/usr/local/hadoop-3.3.5/bin/hadoop

# Removendo tar.gz
!rm hadoop-3.3.5.tar.gz

A saída de streaming foi truncada nas últimas 5000 linhas.
hadoop-3.3.5/share/doc/hadoop/hadoop-yarn/hadoop-yarn-server/hadoop-yarn-server-nodemanager/apidocs/org/apache/hadoop/yarn/server/nodemanager/containermanager/resourceplugin/com/nvidia/NvidiaGPUPluginForRuntimeV2.NvidiaCommandExecutor.html
hadoop-3.3.5/share/doc/hadoop/hadoop-yarn/hadoop-yarn-server/hadoop-yarn-server-nodemanager/apidocs/org/apache/hadoop/yarn/server/nodemanager/containermanager/resourceplugin/com/nvidia/package-tree.html
hadoop-3.3.5/share/doc/hadoop/hadoop-yarn/hadoop-yarn-server/hadoop-yarn-server-nodemanager/apidocs/org/apache/hadoop/yarn/server/nodemanager/containermanager/resourceplugin/com/nvidia/package-frame.html
hadoop-3.3.5/share/doc/hadoop/hadoop-yarn/hadoop-yarn-server/hadoop-yarn-server-nodemanager/apidocs/org/apache/hadoop/yarn/server/nodemanager/containermanager/resourceplugin/com/nvidia/NvidiaGPUPluginForRuntimeV2.html
hadoop-3.3.5/share/doc/hadoop/hadoop-yarn/hadoop-yarn-server/hadoop-yarn-ser

In [ ]:
# Adicionando JAVA_HOME ao hadoop-env.sh
!sed -i '/export JAVA_HOME=/a export JAVA_HOME=\/usr\/lib\/jvm\/java-1.11.0-openjdk-amd64' /usr/local/hadoop-3.3.5/etc/hadoop/hadoop-env.sh

# Variável home do Hadoop
os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.3.5"

## Python mapper.py e reducer.py

A ideia central do MapReduce é o processamento paralelo. Mesmo em um ambiente local simulado pelo Hadoop Streaming, haverá um certo nível de paralelismo, embora limitado pelos recursos da sua máquina.

O Hadoop Streaming cria múltiplos processos para simular os diferentes mappers. Cada processo irá processar uma parte dos dados de entrada e o Hadoop Streaming gerencia a execução dessas tarefas de forma similar a um cluster Hadoop completo, distribuindo o trabalho entre os processos simulados.

#### Mapper e Reducer (.py)

In [ ]:
%%writefile mapper.py
#!/usr/bin/env python
import sys

for line in sys.stdin:
    line = line.strip()
    try:
        value = int(line)
        print('%s\t%s' % ('total', value))
    except ValueError:
        # Skip non-numeric lines
        pass

Writing mapper.py


In [ ]:
%%writefile reducer.py
#!/usr/bin/env python
import sys

current_key = None
total = 0

for line in sys.stdin:
    line = line.strip()
    key, value = line.split('\t', 1)

    try:
        value = int(value)
    except ValueError:
        continue

    if current_key == key:
        total += value
    else:
        if current_key:
            print('%s\t%s' % (current_key, total))
        current_key = key
        total = value

# Print the final result
if current_key:
    print('%s\t%s' % (current_key, total))

Writing reducer.py


## Gera dados

In [ ]:
import random

def generate_random_numbers(n):
  """
  Gera n valores inteiros aleatórios entre 0 e 1000 e salva em um arquivo.

  Args:
    n: O número de valores a serem gerados.
  """
  with open('numbers.txt', 'w') as f:
    for _ in range(n):
      random_number = random.randint(0, 1000)
      f.write(str(random_number) + '\n')

generate_random_numbers(1000)


## Execução

Aqui, os 3 arquivos serão processados em paralelo.

In [ ]:
# argumento -input: diretório contendo os arquivos para leitura
# argumento -output: diretório onde os arquivos de saída serão gravados

!/content/hadoop-3.3.5/bin/hadoop jar /content/hadoop-3.3.5/share/hadoop/tools/lib/hadoop-streaming-3.3.5.jar  \
  -input /content/numbers.txt /content/numbers.txt /content/numbers.txt \
  -output /content/output_hadoop_execution_numbers_2 \
  -mapper 'python mapper.py' \
  -reducer 'python reducer.py'

2024-09-10 21:17:12,908 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2024-09-10 21:17:13,169 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2024-09-10 21:17:13,169 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2024-09-10 21:17:13,200 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2024-09-10 21:17:13,636 INFO mapred.FileInputFormat: Total input files to process : 3
2024-09-10 21:17:13,685 INFO mapreduce.JobSubmitter: number of splits:3
2024-09-10 21:17:14,121 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1898762194_0001
2024-09-10 21:17:14,121 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-09-10 21:17:14,493 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2024-09-10 21:17:14,496 INFO mapreduce.Job: Running job: job_local1898762194_0001
2024-09-10 21:17:14,516 INFO mapred.LocalJobRunner: OutputCommitter set in config null
2024-09

## Resultado

In [ ]:
# Arquivos de saída (informe o novo caminho output, caso tenha alterado)
!ls /content/output_hadoop_execution_numbers_2

part-00000  _SUCCESS


In [ ]:
# Primeiras 50 linhas (informe o novo caminho output, caso tenha alterado)
!head -200 /content/output_hadoop_execution_numbers_2/part-00000

total	1506150
